In [ ]:
!pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
import pandas as pd
from datasets import load_dataset
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
import torch

DEVICE = torch.device("cuda:0")

# Загрузка датасета с использованием trust_remote_code=True
dataset = load_dataset('IlyaGusev/gazeta', revision="v2.0", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/12.2k [00:00<?, ?B/s]

gazeta.py:   0%|          | 0.00/2.98k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

gazeta_train.jsonl:   0%|          | 0.00/550M [00:00<?, ?B/s]

gazeta_val.jsonl:   0%|          | 0.00/56.1M [00:00<?, ?B/s]

gazeta_test.jsonl:   0%|          | 0.00/61.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/60964 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6793 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6369 [00:00<?, ? examples/s]

In [ ]:
# Функция для создания пар "текст новости" и "заголовок"
def prepare_data(examples):
    inputs = [f"Текст новости: {text}\nЗаголовок:" for text in examples['text']]
    return {'input_text': inputs}

# Применение функции к датасету
dataset = dataset.map(prepare_data, batched=True)

Map:   0%|          | 0/60964 [00:00<?, ? examples/s]

Map:   0%|          | 0/6793 [00:00<?, ? examples/s]

Map:   0%|          | 0/6369 [00:00<?, ? examples/s]

In [ ]:
# Функция для токенизации данных
def tokenize_function(examples):
    return tokenizer(examples['input_text'], padding="max_length", truncation=True, max_length=128)

# Инициализация токенизатора
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")

# Применение токенизации к датасету
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Добавление столбца labels
def add_labels(examples):
    examples["labels"] = examples["input_ids"].copy()
    return examples

tokenized_dataset = tokenized_dataset.map(add_labels, batched=True)

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/574 [00:00<?, ?B/s]

Map:   0%|          | 0/60964 [00:00<?, ? examples/s]

Map:   0%|          | 0/6793 [00:00<?, ? examples/s]

Map:   0%|          | 0/6369 [00:00<?, ? examples/s]

Map:   0%|          | 0/60964 [00:00<?, ? examples/s]

Map:   0%|          | 0/6793 [00:00<?, ? examples/s]

Map:   0%|          | 0/6369 [00:00<?, ? examples/s]

In [ ]:
# Разделение датасета на обучающую и валидационную выборки
train_dataset = tokenized_dataset['train'].select(range(1000))  # Используем только первые 1000 примеров
val_dataset = tokenized_dataset['train'].select(range(1000, 1200))  # Используем следующие 200 примеров для валидации

In [ ]:
# Загрузка модели
model = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2").to(DEVICE)

# Параметры обучения
training_args = TrainingArguments(
    output_dir="./finetuned",       # директория с выходными данными
    overwrite_output_dir=True,      # перезаписываем содержимое выходной директории при каждом запуске
    num_train_epochs=3,             # число эпох обучения
    per_device_train_batch_size=4,  # batch size для обучения
    per_device_eval_batch_size=4,   # batch size для выполнения
    warmup_steps=500,               # количество шагов для "прогрева" (управление скоростью обучения)
    weight_decay=0.01,              # регуляризация весов
    logging_dir='./logs',           # директория для логов
    logging_steps=10,
    save_steps=10_000,
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=500,
    fp16=True,                      # использование 16-битной точности
    remove_unused_columns=False,    # не удалять неиспользуемые столбцы
)

# Инициализация класса обучения
trainer = Trainer(
    model=model,                 # модель
    args=training_args,          # параметры обучения
    train_dataset=train_dataset, # датасет для обучения
    eval_dataset=val_dataset,    # датасет для валидации
    optimizers=(torch.optim.AdamW(model.parameters(), lr=5e-5), None) # оптимизатор
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Обучение модели
trainer.train()

# Сохранение модели
trainer.save_model("./finetuned")
tokenizer.save_pretrained("./finetuned")

Step,Training Loss,Validation Loss
500,2.712700,2.891506


('./finetuned/tokenizer_config.json',
 './finetuned/special_tokens_map.json',
 './finetuned/vocab.json',
 './finetuned/merges.txt',
 './finetuned/added_tokens.json')

In [ ]:
# Загрузка обученной модели и токенизатора
model = GPT2LMHeadModel.from_pretrained("./finetuned").to(DEVICE)
tokenizer = GPT2Tokenizer.from_pretrained("./finetuned")

In [ ]:
# Примеры текстов новостей
example_texts = [
    "Вчера вечером в центре города произошло ДТП с участием двух автомобилей. По предварительным данным, никто не пострадал, но машины получили значительные повреждения.",
    "Вчера в парке был проведен фестиваль цветов, на котором присутствовали более тысячи гостей. Фестиваль прошел успешно, и все остались довольны.",
    "Вчера в городе был зафиксирован рекордный уровень осадков. Дождь продолжался более 12 часов, что привело к затоплению нескольких улиц.",
    "Вчера в театре состоялась премьера нового спектакля, который получил высокие оценки критиков и зрителей. Спектакль прошел с аншлагом.",
    "Вчера в городе был проведен массовый флешмоб, посвященный Дню города. Тысячи людей приняли участие в праздновании, и все остались довольны.",
    "Вчера в городе был проведен марафон, в котором приняли участие более тысячи спортсменов. Марафон прошел успешно, и все участники получили медали.",
    "Вчера в городе был проведен фестиваль музыки, на котором выступили более десяти групп. Фестиваль прошел успешно, и все остались довольны.",
    "Вчера в городе был проведен фестиваль кино, на котором были показаны более десяти фильмов. Фестиваль прошел успешно, и все остались довольны.",
    "Вчера в городе был проведен фестиваль танцев, на котором выступили более десяти коллективов. Фестиваль прошел успешно, и все остались довольны.",
    "Вчера в городе был проведен фестиваль искусств, на котором были представлены работы более десяти художников. Фестиваль прошел успешно, и все остались довольны."
]

# Генерация заголовков
for example_text in example_texts:
    input_ids = tokenizer.encode(f"Текст новости: {example_text}\nЗаголовок:", return_tensors="pt").to(DEVICE)
    output = model.generate(input_ids, max_length=50, num_return_sequences=1, temperature=0.7, top_k=50, top_p=0.95, do_sample=True)
    generated_title = tokenizer.decode(output[0], skip_special_tokens=True)
    print(f"Текст новости: {example_text}")
    print(f"Сгенерированный заголовок: {generated_title}")
    print()

Текст новости: Вчера вечером в центре города произошло ДТП с участием двух автомобилей. По предварительным данным, никто не пострадал, но машины получили значительные повреждения.
Сгенерированный заголовок: Текст новости: Вчера вечером в центре города произошло ДТП с участием двух автомобилей. По предварительным данным, никто не пострадал, но машины получили значительные повреждения.
Заголовок: Исламисты взорвали в Париже «Батальон смерти

Текст новости: Вчера в парке был проведен фестиваль цветов, на котором присутствовали более тысячи гостей. Фестиваль прошел успешно, и все остались довольны.
Сгенерированный заголовок: Текст новости: Вчера в парке был проведен фестиваль цветов, на котором присутствовали более тысячи гостей. Фестиваль прошел успешно, и все остались довольны.
Заголовок: Как избежать стресса в школе?
Параметры стресса в школе:


Текст новости: Вчера в городе был зафиксирован рекордный уровень осадков. Дождь продолжался более 12 часов, что привело к затоплению нескольких